In [7]:
import sys
sys.path.append('..')
from utils import get_mongo_client, test_mongo_client, get_customers_database_infos, get_all_database_names
from bson.timestamp import Timestamp
from datetime import datetime
import pandas as pd
import random
import numpy as np
from pathlib import Path

#client_local = get_mongo_client("local")
client_backup = get_mongo_client("backup")
client_prod = get_mongo_client("production")
#test_mongo_client(client_local)

In [8]:
# get the list of customer databases
# for each c database get:

In [9]:
# end result
# table:
# | customer_name | db_name | space | license model | users_limit | total_users | restricted_users |
# | |   

In [10]:
infos = get_customers_database_infos(client_backup)

df_infos = pd.DataFrame(infos)
sel_col = ["name", "dbName", "hostnames", "licenseModel", "licensePeriods", "renewalDate", "accountsLimit"]
df_infos = df_infos[sel_col]
df_infos = df_infos.rename(columns={
    'name': 'customer_name', 
    'dbName': 'db_name',
    'hostnames': 'host_name',
    'licenseModel': 'license_model',
    'licensePeriods': 'license_periods',
    'renewalDate': 'renewal_date',
    'accountsLimit': 'user_limit'
})
# hostnames is the ID pf df_infos

In [11]:
df_infos

,customer_name,db_name,host_name,license_model,license_periods,renewal_date,user_limit
0,afpa,org_afpa-beta,afpa-beta.didask.com,accounts,[],NaT,8.0
1,alliance-emploi,org_alliance-emploi,alliance-emploi.didask.com,accounts,[],NaT,250.0
2,anfh-corse,org_anfh-corse,anfh-corse.didask.com,accounts,[],NaT,250.0
3,anfh-bretagne,org_anfh-bretagne,anfh-bretagne.didask.com,accounts,[],NaT,NaN
4,Apside,org_apside,apside-beta.didask.com,accounts,"[{'start': 2023-11-29 23:00:00, 'end': 2024-11...",NaT,50.0
...,...,...,...,...,...,...,...
172,Test | SUP'EXPERTISE,sandbox_org_academie_supexpertise,test.supexpertise.didask.com,accounts,[],NaT,3000.0
173,Voltaïca,org_voltaica,voltaica.didask.com,accounts,NaN,NaT,0.0
174,LegalVision,org_legalvision,legalvision.didask.com,accounts,NaN,NaT,0.0
175,Ecole Vendôme,org_ecole_vendome,ecole-vendome.didask.com,accounts,NaN,NaT,100.0


In [12]:
#client = client_local
client = client_backup
#client= client_prod

user_data = []

for i in range(0, len(df_infos)):
    customer_name = df_infos.iloc[i]["customer_name"]
    db_name = df_infos.iloc[i]["db_name"]
    hostname = df_infos.iloc[i]["host_name"]
    account_limit = df_infos.iloc[i]["user_limit"]

    db = client[db_name]
    user_collection = db['user']

    #print(f'customer: {customer_name}')
    #print(f'hostname: {hostname}')
    #print(f'account limit: {account_limit}')
    #print('---')

    for user in user_collection.find({}, {'isStandard': 1, 'disabledDate': 1, '_id': 0}):
        user["host_name"] = hostname
        for c in ['isStandard', 'disabledDate']:
            user[c] = user.get(c, None)
        user_data.append(user)

df_user = pd.DataFrame(user_data)
filtered_df = df_user[df_user['disabledDate'].isna()]

def aggregate_data(group):
    nb_users_total = len(group)
    nb_users_active = group['isStandard'].fillna(True).sum()
    nb_users_restricted = (~group["isStandard"].fillna(True)).sum()
    return pd.Series({
        'nb_users_total': nb_users_total,
        'nb_users_active': nb_users_active,
        'nb_users_restricted': nb_users_restricted
    })

In [13]:
df_user_group = filtered_df.groupby("host_name").apply(aggregate_data).reset_index()
df_user_group

,host_name,nb_users_total,nb_users_active,nb_users_restricted
0,academie-supexpertise.didask.com,1208,1160,48
1,adonnante.didask.com,7,7,0
2,afpa-beta.didask.com,40,27,13
3,afpa-formation.didask.com,20,18,2
4,afpa-sandbox.didask.com,6,4,2
...,...,...,...,...
172,voltaica.didask.com,2,2,0
173,votre-espace.didask.com,74,68,6
174,wisper.didask.com,41,38,3
175,yeap.didask.com,300,296,4


In [ ]:
df_res = df_infos.merge(df_user_group, how='left', on='host_name')
df_res = df_res.sort_values('nb_users_restricted', ascending=False).reset_index(drop=True)
df_res['user_limit'] = df_res['user_limit'].replace({0: np.nan})
df_res['user_limit'] = df_res['user_limit'].astype('Int64')
df_res['license_periods'] = df_res['license_periods'].apply(lambda x: np.nan if x == [] else x)
df_res['nb_over_limit'] = df_res['nb_users_total'] - df_res['user_limit']

In [ ]:
df_res = df_res.sort_values("nb_over_limit", ascending=False)

In [ ]:
missing_user_limit = df_res["user_limit"].isna().sum()
print(missing_user_limit)

missing_license_period = df_res["license_periods"].isna().sum()
print(missing_license_period)

In [ ]:
## ~177 clients
## 50 missing info user limit
## 35 missing info on license period

In [ ]:
df_res['is_over_limit'] = df_res["nb_over_limit"] > 0
df_res['is_over_limit_because_restricted'] = (df_res['is_over_limit'] & (df_res['nb_users_restricted'] >= df_res['nb_over_limit']))
df_res['is_missing_limit'] = df_res["user_limit"].isna()

# set to False when user_limit is not available
df_res['is_over_limit'] = df_res['is_over_limit'].fillna(False)
df_res['is_over_limit_because_restricted'] = df_res['is_over_limit_because_restricted'].fillna(False)

In [ ]:
nb_customers = len(df_res)
nb_over_limit = df_res["is_over_limit"].sum()
nb_over_limit_because_restricted = df_res['is_over_limit_because_restricted'].sum()
nb_missing_limit = df_res['is_missing_limit'].sum()

In [ ]:
print(f'nb customers: {nb_customers}')
print(f'nb over limit: {nb_over_limit}')
print(f'nb over limit because of restricted users: {nb_over_limit_because_restricted}')
print(f'nb missing limit: {nb_missing_limit}')

In [ ]:
def highlight_rows(dataframe, conditions):
    """
    Apply multiple highlighting conditions to a DataFrame.
    
    :param dataframe: pandas DataFrame
    :param conditions: List of tuples, each containing the column, condition function, and color
    :return: Styled DataFrame
    """
    def highlight(row):
        styles = [''] * len(row)  # Default style (no background color)
        for column, condition, color in conditions:
            if condition(row[column]):
                styles = [f'background-color: {color}; color: black' for _ in row]
        return styles

    styled_df = dataframe.style.apply(highlight, axis=1)
    return styled_df

In [ ]:
conditions = [
    ('is_over_limit', lambda val: val, '#FFB266'),
    ('is_over_limit_because_restricted', lambda val: val, '#FF9999'),
    ('is_missing_limit', lambda val: val, '#FFFF99')
]

In [ ]:
styled_df = highlight_rows(df_res, conditions)
styled_df

In [ ]:

path_excel_output = Path('~/Desktop/nb-users-per-customer.xlsx')
name_sheet_1 = "Clients ordonnés par dépassement de licence"
name_sheet_2 = "Clients ordonnés par nombre d'utilisateurs restreints"
with pd.ExcelWriter(path_excel_output, engine='openpyxl') as writer:
    styled_df.to_excel(writer, sheet_name=name_sheet_1, index=False)
    #df_res.sort_values('nb_users_restricted', ascending=False).to_excel(writer, sheet_name=name_sheet_2, index=False)